# k_{inf} в зависимости от плотности теплоносителя, температуры теплоносителя, температуры топлива и концентрации борной кислоты

## 1. Загрузка данных

In [1]:
import pandas as pd

Загрузка данных

In [2]:
df = pd.read_excel('Input/Data.xlsx')
df.head(3)

,\rho(g/cm^3),T_c(K),T_f(K),c_b(ppm),3S_{tr1}(1),3S_{tr2}(2),\Sigma_{a1}(3),\Sigma_{a2}(4),nuS_{f1}(5),nuS_{f2}(6),\Sigma_{f1}(7),\Sigma_{f2}(8),\Sigma_{1->2}(9),k_{inf}(10),\sigma_{a2}^{Xe},\sigma_{a2}^{Sm}
0,0.7235,575.0,1027.0,600.0,0.661853,2.568298,0.009376,0.085584,0.006677,0.133250,0.002620,0.054750,0.017392,1.261041,1072970.0,35421.398
1,0.7750,448.0,1043.0,1500.0,0.679602,2.874687,0.009581,0.098339,0.006723,0.139256,0.002636,0.057218,0.018726,1.174265,1141000.0,36147.801
2,0.9125,370.5,1418.0,750.0,0.727336,3.668742,0.009908,0.102815,0.006844,0.153823,0.002681,0.063203,0.022463,1.249619,1308480.0,39214.000


In [3]:
names = ['\\rho(g/cm^3)',
 'T_c(K)',
 'T_f(K)',
 'c_b(ppm)',
 'k_{inf}(10)']

In [4]:
df = df[names]
df.head(3)

,\rho(g/cm^3),T_c(K),T_f(K),c_b(ppm),k_{inf}(10)
0,0.7235,575.0,1027.0,600.0,1.261041
1,0.7750,448.0,1043.0,1500.0,1.174265
2,0.9125,370.5,1418.0,750.0,1.249619


### 2 Нормализация признаков

In [5]:
features = ['\\rho(g/cm^3)',
 'T_c(K)',
 'T_f(K)',
 'c_b(ppm)']

In [6]:
df_features_norm = df.copy()

In [7]:
for i in range(len(features)):
    df_features_norm[features[i]] = (df[features[i]]-df[features[i]].mean())/df[features[i]].std()

In [8]:
df_features_norm

,\rho(g/cm^3),T_c(K),T_f(K),c_b(ppm),k_{inf}(10)
0,-0.316426,1.416398,-0.039094,-1.035019,1.261041
1,0.007922,-0.005506,-0.001946,0.006984,1.174265
2,0.873902,-0.873204,0.868693,-0.861352,1.249619
3,-0.858058,0.862191,-0.872586,0.875321,1.127466
4,-0.425067,-0.439355,0.433373,-1.295520,1.280245
...,...,...,...,...,...
252,-0.844527,-0.886762,-1.539169,-0.603564,1.243578
253,0.887433,0.848634,0.202111,1.133108,1.067387
254,0.021453,-0.019064,1.072750,-1.471900,1.301615
255,-1.710507,1.716332,-0.668529,0.264771,1.152782


In [9]:
df_features_norm.sample(frac=1) 

,\rho(g/cm^3),T_c(K),T_f(K),c_b(ppm),k_{inf}(10)
83,-0.452130,1.268925,1.440051,-0.345778,1.185053
84,-0.019140,-0.032622,-1.607187,0.088390,1.190173
95,-1.534604,1.485849,-1.389527,1.607979,1.100686
127,-1.696975,0.563921,0.079676,-0.400047,1.178676
89,0.630345,1.052000,-0.083568,0.305474,1.142945
...,...,...,...,...,...
179,-0.546846,0.550362,1.045542,1.051702,1.092039
86,0.846840,-0.900320,1.004731,0.956727,1.075586
145,0.373258,0.604594,0.229316,-1.390494,1.308652
124,-0.830995,-0.303777,0.950315,0.468289,1.131797


In [10]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(df_features_norm[['\\rho(g/cm^3)', 'T_c(K)', 'T_f(K)', 'c_b(ppm)']],
                                                    df_features_norm[['k_{inf}(10)']],
                                                    test_size=0.33,
                                                    random_state=42)

# Нейросетевые модели

In [11]:
%%time

# Use scikit-learn to grid search the batch size and epochs
import numpy
from sklearn.model_selection import GridSearchCV
from keras.models import Sequential
from keras.layers import Dense
from keras.wrappers.scikit_learn import KerasRegressor
import tensorflow as tf

# Function to create model, required for KerasRegressor
def create_model(activation='relu', learn_rate=0.01, momentum=0):
    # create model
    model = Sequential()
    model.add(Dense(4, input_dim=4, activation=activation))
    model.add(Dense(1, kernel_initializer='normal'))
    
    optimizer = tf.keras.optimizers.RMSprop(learning_rate = learn_rate, rho = 0.99, momentum = momentum, epsilon=1e-10)
    
    # Compile model
    model.compile(loss='mse', optimizer=optimizer)
    return model

X = X_train
Y = y_train

# create model
model = KerasRegressor(build_fn=create_model, verbose=0)

# define the grid search parameters
batch_size = [10, 15, 20]
epochs = [1000, 1200, 1300]
activation = ['tanh', 'sigmoid', 'linear']
learn_rate = [0.00001, 0.00005, 0.00009]
momentum = [0]

param_grid = dict(batch_size=batch_size,
                  epochs=epochs,
                  activation=activation,
                  learn_rate=learn_rate,
                  momentum=momentum
                 )
grid = GridSearchCV(estimator=model, param_grid=param_grid, n_jobs=5)
grid_result = grid.fit(X, Y)

# summarize results
print("Best: %f using %s" % (grid_result.best_score_, grid_result.best_params_))
means = grid_result.cv_results_['mean_test_score']
stds = grid_result.cv_results_['std_test_score']
params = grid_result.cv_results_['params']
for mean, stdev, param in zip(means, stds, params):
    print("%f (%f) with: %r" % (mean, stdev, param))

Best: -0.000326 using {'activation': 'sigmoid', 'batch_size': 10, 'epochs': 1300, 'learn_rate': 9e-05, 'momentum': 0}
-1.054464 (0.034671) with: {'activation': 'tanh', 'batch_size': 10, 'epochs': 1000, 'learn_rate': 1e-05, 'momentum': 0}
-0.004923 (0.002239) with: {'activation': 'tanh', 'batch_size': 10, 'epochs': 1000, 'learn_rate': 5e-05, 'momentum': 0}
-0.000835 (0.000652) with: {'activation': 'tanh', 'batch_size': 10, 'epochs': 1000, 'learn_rate': 9e-05, 'momentum': 0}
-1.002602 (0.020865) with: {'activation': 'tanh', 'batch_size': 10, 'epochs': 1200, 'learn_rate': 1e-05, 'momentum': 0}
-0.003628 (0.002092) with: {'activation': 'tanh', 'batch_size': 10, 'epochs': 1200, 'learn_rate': 5e-05, 'momentum': 0}
-0.000374 (0.000108) with: {'activation': 'tanh', 'batch_size': 10, 'epochs': 1200, 'learn_rate': 9e-05, 'momentum': 0}
-0.930103 (0.035946) with: {'activation': 'tanh', 'batch_size': 10, 'epochs': 1300, 'learn_rate': 1e-05, 'momentum': 0}
-0.002601 (0.001671) with: {'activation': 

In [12]:
-0.000326 using {'activation': 'sigmoid', 'batch_size': 10, 'epochs': 1300, 'learn_rate': 9e-05, 'momentum': 0}

SyntaxError: invalid syntax (<ipython-input-12-cfa35a7ac4c0>, line 1)

## Модель 1

In [19]:
def create_model():

    # create model
    model = Sequential()
    model.add(Dense(4, input_dim=4, activation='sigmoid'))
    model.add(Dense(1, kernel_initializer='normal'))

    optimizer = tf.keras.optimizers.RMSprop(learning_rate = 9e-04, rho = 0.99, momentum = 0, epsilon=1e-10)
    
    # Compile model
    model.compile(loss='mse', optimizer=optimizer)
    return model

In [20]:
model = create_model()
model.summary()

Model: "sequential_2"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_4 (Dense)              (None, 4)                 20        
_________________________________________________________________
dense_5 (Dense)              (None, 1)                 5         
Total params: 25
Trainable params: 25
Non-trainable params: 0
_________________________________________________________________


In [21]:
# Обучение модели

EPOCHS = 1300
batch_size = 10

history = model.fit(
  X, Y,
  epochs=EPOCHS, batch_size = batch_size,verbose=0)

In [22]:
hist = pd.DataFrame(history.history)
hist['epoch'] = history.epoch
hist.tail(10)

,loss,epoch
1290,0.000007,1290
1291,0.000007,1291
1292,0.000011,1292
1293,0.000011,1293
1294,0.000013,1294
1295,0.000009,1295
1296,0.000012,1296
1297,0.000009,1297
1298,0.000008,1298
1299,0.000009,1299


In [23]:
import statsmodels as statsmodels
import statsmodels.api as sm

RMSPE = statsmodels.tools.eval_measures.rmspe(y_test.values, model.predict(X_test))
RMSPE

array([0.02699464])

In [24]:
import statsmodels as statsmodels
import statsmodels.api as sm

RMSPE = statsmodels.tools.eval_measures.rmspe(y_train.values, model.predict(X_train))
RMSPE

array([0.02185604])